In [1]:
import sys
import os
sys.path.append("../")
sys.path.append("../..")

from KETIPrePartialDataPreprocessing import main

inputType ='file' # or file
refine_param = {'removeDuplication':True, 'staticFrequency':True}
outlier_param= {'certainOutlierToNaN':True, 'uncertainOutlierToNaN':True, 'data_type':'air'}
imputation_param ={
"imputation_method":[
    {"min":0,"max":2,"method":"most_frequent"}],"totalNanLimit":30}

# mean, median X
# ffill~polynomial_interpolate=> 각 구간마다 딱 고만큼만 Fill

input_data = main.inputControl(inputType)
from KETIPrePartialDataPreprocessing import data_preprocessing
import numpy as np
data = input_data.replace(4.0, np.nan)
data = data[:1000]

In [2]:
from sklearn.impute import SimpleImputer
data2 = SimpleImputer(strategy='mean').fit_transform(data)


In [3]:
import pandas as pd
class PandasSimpleImputer(SimpleImputer):
    def __init__(self, strategy, missing_values):
        super().__init__(strategy = strategy, missing_values = missing_values)

    def fit(self, X, y=None):
        self.columns = X.columns
        return super().fit(X, y)
    def transform(self, X):
        return pd.DataFrame(super().transform(X), columns = self.columns)

def simpleMethod(data, method, max):
    data = PandasSimpleImputer(strategy=method, missing_values = np.nan).fit_transform(data)
data2 =data [['pm10']]
data3 = PandasSimpleImputer(strategy='most_frequent', missing_values = np.NaN).fit_transform(data2) 

In [4]:
data2

,pm10
timedate,
2021-01-28 12:51:00,-9999.0
2021-01-28 12:52:00,64.0
2021-01-28 12:53:00,56.0
2021-01-28 12:54:00,59.0
2021-01-28 12:55:00,59.0
...,...
2021-01-29 05:28:00,48.0
2021-01-29 05:29:00,48.0
2021-01-29 05:30:00,48.0


In [5]:
from KETIPrePartialDataPreprocessing import data_preprocessing

MDP = data_preprocessing.DataPreprocessing()
refined_data = MDP.get_refinedData(data, refine_param)
datawithMoreCertainNaN, datawithMoreUnCertainNaN = MDP.get_outlierToNaNData(refined_data, outlier_param)
imputed_data = MDP.get_imputedData(datawithMoreUnCertainNaN, imputation_param)

data Length after removeDuplication: 1000
data Length after make_static_frequency: 1002
NaN Ratio: temp 8.28 %
most_frequent
Couldn't find a proper imputation method.
NaN Ratio: co2 3.29 %
most_frequent
Couldn't find a proper imputation method.
NaN Ratio: pm10 0.4 %
most_frequent
Couldn't find a proper imputation method.
                     temp  co2  pm10
timedate                            
2021-01-28 12:51:00   3.0  5.0   NaN
2021-01-28 12:52:00   3.0  5.0  64.0
2021-01-28 12:53:00   NaN  6.0  56.0
2021-01-28 12:54:00   NaN  7.0  59.0
2021-01-28 12:55:00   NaN  7.0  59.0
...                   ...  ...   ...
2021-01-29 05:28:00   0.0  1.0  48.0
2021-01-29 05:29:00   0.0  1.0  48.0
2021-01-29 05:30:00   0.0  1.0  48.0
2021-01-29 05:31:00   0.0  1.0  48.0
2021-01-29 05:32:00   0.0  1.0  48.0

[1002 rows x 3 columns]


In [6]:
def getConsecutiveNaNInfoOverThresh(data, column_name, thresh):
    # column data: data with one column
    # thresh: Threshold value to select specific ranges
    
    a = data.index
    b = data[column_name].values
    idx0 = np.flatnonzero(np.r_[True, np.diff(np.isnan(b))!=0,True])
    count = np.diff(idx0)
    idx = idx0[:-1]
    valid_mask = (count>=thresh) & np.isnan(b[idx])
    out_idx = idx[valid_mask]
    out_num = a[out_idx]
    out_count = count[valid_mask]
    out = zip(out_num, out_count)
    return out

def setNaNSpecificDuration(data, column_name, NaNInfoOverThresh, tresh):
    for NaNInfoOverThreshitem in NaNInfoOverThresh:
        indexLocation = data.index.get_loc(NaNInfoOverThreshitem[0])
        consecutiveNum= NaNInfoOverThreshitem[1]
        data.loc[indexLocation+thresh:indexLocation+consecutiveNum, column_name] = np.nan
    return data


In [7]:
a = data.index
b = data.values

b

array([[ 3.000e+00,  5.000e+00, -9.999e+03],
       [ 3.000e+00,  5.000e+00,  6.400e+01],
       [       nan,  6.000e+00,  5.600e+01],
       ...,
       [ 0.000e+00,  1.000e+00,  4.800e+01],
       [ 0.000e+00,  1.000e+00,  4.800e+01],
       [ 0.000e+00,  1.000e+00,  4.800e+01]])

In [8]:
column_name='in_ciai'
data =datawithMoreCertainNaN  
thresh = 3

NaNInfoOverThresh = list(getConsecutiveNaNInfoOverThresh(data, column_name, thresh))
DataWithMaskedNaN = setNaNSpecificDuration(data, column_name, NaNInfoOverThresh, thresh)

KeyError: 'in_ciai'

In [ ]:
NaNInfoOverThresh[0][0]

In [ ]:
data.loc[indexLocation+thresh:indexLocation+consecutiveNum, column_name] = 0

In [ ]:
data[indexLocation+thresh:indexLocation+consecutiveNum]

In [ ]:
count[valid_mask]

In [ ]:
flat